In [ ]:
#mount drive
%cd ..
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive

# list the contents of /mydrive
!ls /mydrive

#Navigate to /mydrive/yolov4
%cd /mydrive/Ocular_Disease/

In [ ]:
!pip install tensorflow-addons

In [ ]:
import os
import shutil
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from PIL import Image
from tensorflow_addons.metrics import F1Score  # Import the F1Score metric from TensorFlow Addons
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array

# Register the custom object
custom_objects = {'F1Score': F1Score}

# Load the saved Keras model
with tf.keras.utils.custom_object_scope(custom_objects):
    model = load_model('./ocular_disease_model.h5')

##  Classification

In [ ]:
# Define the desired image size for model input
image_size = (224, 224)

# Define the class labels
class_labels = ['cataract', 'glaucoma', 'hyper', 'myopia', 'age', 'other']

# Set the directory where the test images are stored
test_data_dir = '/content/gdrive/MyDrive/Ocular_Disease/sd_outputs'

# Create directories for storing images by their predicted class labels
for label in class_labels:
    label_dir = os.path.join(test_data_dir, 'results', label)
    tf.io.gfile.makedirs(label_dir)  # Ensure the directory exists for each label

# Iterate over each class folder
for class_label in class_labels:
    # Get all images from the class directory
    class_dir = os.path.join(test_data_dir, class_label)
    test_images = os.listdir(class_dir)  # Get all image filenames from the current class folder

    # Process each test image, make predictions, and save them in the corresponding directory
    for image_file in test_images:
        # Load and preprocess the image
        image_path = os.path.join(class_dir, image_file)
        image = load_img(image_path, target_size=image_size)  # Resize image
        image_array = img_to_array(image)  # Convert image to an array
        image_tensor = tf.expand_dims(image_array, 0)  # Add batch dimension

        # Get model's prediction for the image
        outputs = model.predict(image_tensor)
        predicted = tf.argmax(outputs, axis=1)  # Get index of predicted class
        predicted_label = class_labels[predicted.numpy()[0]]  # Get the label name based on prediction

        # Print the prediction result for reference
        print(f'Image: {image_file}, Predicted class: {predicted_label}')

        # Move the image to the corresponding folder for its predicted class
        destination_dir = os.path.join(test_data_dir, 'result_cataract', predicted_label)
        destination_path = os.path.join(destination_dir, image_file)  # Define the path to save the image
        shutil.copyfile(image_path, destination_path)  # Copy image to the appropriate folder


## For Mapping

In [ ]:
from tensorflow.keras.preprocessing import image
from sklearn.metrics import confusion_matrix

# Specify the path to the test images directory
test_dir = '/content/gdrive/MyDrive/Ocular_Disease/sd_outputs/try'  # Directory where test images are stored

# Load the dataset CSV file which contains the true labels for the test images
dataset_file = '/content/gdrive/MyDrive/Ocular_Disease/true_labels.csv'
df = pd.read_csv(dataset_file)  # Read the CSV into a DataFrame

# Extract the image filenames and true labels from the dataset
image_files = df['image'].values  # List of image filenames
true_labels = df['true_label'].values  # List of corresponding true labels

# Create lists to store predictions and predicted labels
predictions = []  # List to store model's raw predictions
predicted_labels = []  # List to store the predicted class labels

# Perform inference for each image in the directory
for image_file in image_files:
    # Load and preprocess the test image
    image_path = os.path.join(test_dir, image_file)  # Get the full image path
    img = image.load_img(image_path, target_size=(224, 224))  # Load and resize the image
    x = image.img_to_array(img)  # Convert image to array
    x = np.expand_dims(x, axis=0)  # Add batch dimension

    # Perform the inference (predict the class for the image)
    prediction = model.predict(x)  # Get the model's output
    predicted_label_index = np.argmax(prediction)  # Get index of the predicted class (class with highest probability)
    predicted_label = class_labels[predicted_label_index]  # Convert index to class label

    # Append the prediction and predicted label to the respective lists
    predictions.append(prediction)  # Store raw prediction
    predicted_labels.append(predicted_label)  # Store predicted class label

# Convert the lists to numpy arrays for easier processing later
predictions = np.array(predictions)  # Convert list of raw predictions to numpy array
predicted_labels = np.array(predicted_labels)  # Convert list of predicted labels to numpy array

# Compute the confusion matrix comparing true labels vs predicted labels
conf_mat = confusion_matrix(true_labels, predicted_labels)

# Define a function to plot the confusion matrix
def plot_confusion_matrix(cm, classes, normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]  # Normalize the matrix to percentages
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)  # Plot the confusion matrix as an image
    plt.title(title)  # Set the title for the plot
    plt.colorbar()  # Add a color bar
    tick_marks = np.arange(len(classes))  # Set tick marks based on the number of classes
    plt.xticks(tick_marks, classes, rotation=45)  # Set x-axis labels with class names
    plt.yticks(tick_marks, classes)  # Set y-axis labels with class names

    fmt = '.2f' if normalize else 'd'  # Format values as percentage or integer
    thresh = cm.max() / 2.  # Determine threshold for text color based on matrix values
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):  # Loop over matrix cells
        plt.text(j, i, format(cm[i, j], fmt),  # Annotate matrix with values
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")  # Use white or black text for contrast

    plt.tight_layout()  # Ensure everything fits without overlap
    plt.ylabel('True label')  # Label for the y-axis
    plt.xlabel('Predicted label')  # Label for the x-axis

# Plot the confusion matrix for the true vs predicted labels
plt.figure(figsize=(10, 5))  # Set the figure size
plt.grid(False)  # Disable grid lines
plot_confusion_matrix(conf_mat, classes=['normal', 'cataract', 'diab', 'glaucoma', 'hyper', 'myopia', 'age', 'other'], normalize=False)
plt.show()

## Visualization Test

In [ ]:
# Set the path to the directory containing your test images
test_dir = '/content/gdrive/MyDrive/Ocular_Disease/sd_outputs/result_hypertensive'

# Get a list of image file names in the test directory
image_files = os.listdir(test_dir)

In [ ]:
# Calculate the number of rows and columns for subplots based on the number of images
num_images = len(image_files)  # Get the total number of images to display
num_rows = int(np.ceil(np.sqrt(num_images)))  # Calculate the number of rows for the subplots
num_cols = int(np.ceil(num_images / num_rows))  # Calculate the number of columns based on the number of rows

# Create subplots to display the images with appropriate figure size
fig, axes = plt.subplots(num_rows, num_cols, figsize=(18, 18))  # Create a grid of subplots for displaying images

# Perform inference for each image in the directory and display it in the corresponding subplot
for i, image_file in enumerate(image_files):
    # Load and preprocess the test image
    image_path = os.path.join(test_dir, image_file)  # Get the full path to the image file
    img = image.load_img(image_path, target_size=(224, 224))  # Load and resize the image
    x = image.img_to_array(img)  # Convert the image to an array format
    x = np.expand_dims(x, axis=0)  # Add batch dimension for model input

    # Perform inference to get the model's prediction for the image
    predictions = model.predict(x)  # Get the prediction probabilities from the model
    predicted_label_index = np.argmax(predictions)  # Find the index of the class with the highest probability
    predicted_label = class_labels[predicted_label_index]  # Get the predicted label name using the index

    # Get the subplot index for the current image (row and column)
    row_idx = i // num_cols  # Determine the row index for the subplot
    col_idx = i % num_cols  # Determine the column index for the subplot

    # Display the image and predicted label in the corresponding subplot
    axes[row_idx, col_idx].imshow(img)  # Display the image in the subplot
    axes[row_idx, col_idx].set_title(f'Predicted: {predicted_label}')  # Set the title with the predicted label
    axes[row_idx, col_idx].axis('off')  # Turn off axis labels for better visualization

# Adjust the spacing between subplots to avoid overlap
plt.subplots_adjust(hspace=1, wspace=1)  # Adjust horizontal and vertical spacing between subplots

plt.show()